In [1]:
# Machine Learning Algo using Logistic Regression and Factor analysis to predict returns 21 days out
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar
from quantopian.pipeline import CustomFactor
from quantopian.pipeline.factors import Returns
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.filters import Q500US, Q1500US
from quantopian.pipeline.data.zacks import EarningsSurprises

import pandas as pd
import numpy as np
from time import time
import math

import matplotlib.pyplot as plt
from sklearn import ensemble, preprocessing, metrics
from sklearn.linear_model import LogisticRegression

In [2]:
n_fwd_days = 21

In [9]:
bs = morningstar.balance_sheet
cfs = morningstar.cash_flow_statement
is_ = morningstar.income_statement
or_ = morningstar.operation_ratios
er = morningstar.earnings_report
v = morningstar.valuation
vr = morningstar.valuation_ratios

def make_pipeline():
    
    base_universe = Q1500US()
    
    class StdDev(CustomFactor):
        """
        
        Calculate 3 months realized volatility
        
        """
        
        def compute(self, today, asset_ids, out, values):
            # will calculate the column wise standard deviation, ignoring NaNs
            out[:] = np.nanstd(values, axis=0)
            
    class Momentum(CustomFactor):
        # Default inputs
        inputs = [USEquityPricing.close]
        """
        Calculates 1-month price momentum
        1 month closiing price rate of change
        
        Notes High value suggest momentum (shorter term)
        Equivalent to analysis of returns (1-month window)
        
        source:https://www.pnc.com/content/dam/pnc-com/pdf/personal/wealth-investments/WhitePapers/FactorAnalysisFeb2014.pdf
        
        """
        
        # Compute momentum
        def compute(self,today, assets,out,close):
            out[:] = close[-1] / close[0]
            
    class Mean_Reversion_1M(CustomFactor):
        inputs =[Returns(window_length = 21)]
        window_length = 252
        
        def compute(self, today, assets, out, monthly_rets):
            out[:] = (monthly_rets[-1] - np.nanmean(monthly_rets, axis=0)) / \
            np.nanstd(monthly_rets, axis=0)
            
    class Price_Oscillator(CustomFactor):
        """
        4/52-Week Price Oscillator:
        Average close prices over 4 weeks divided by average close prices over 52 weeks all less 1
        source: https://www.math.nyu.edu/faculty/avellane/Lo13030.pdf
        
        Notes:
        A high value suggests momentum
        
        """
        
        inputs = [USEquityPricing.close]
        window_length = 252
        
        def compute(self, today, assets, out, close):
            four_week_period = close[-20:]
            out[:] = (np.nanmean(four_week_period, axis=0) / np.nanmean(close, axis=0))-1
        
    def Earnings_Quality():
        return cfs.operating_cash_flow.latest / \
        EarningsSurprises.eps_act.latest
    
    Price_Momentum_1M = Momentum(window_length=21)
    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=63, mask=base_universe)
    Price_Oscillator = Price_Oscillator()
    
    return Pipeline(
        columns={'pb_ratio': vr.pb_ratio.latest,
                 'gp_ta': is_.gross_profit.latest / bs.total_assets.latest,
                 'roe': or_.roe.latest,
                 'net_margin': or_.net_margin.latest,
                 'assets_turnover': or_.assets_turnover.latest,
                 'gearing': bs.total_debt.latest / bs.total_equity.latest,
                 'forward_earning_yeild': vr.forward_earning_yield.latest,
                 'cf_yield': vr.cf_yield.latest,
                 'dividend_yield': vr.dividend_yield.latest,
                 'market_cap': v.market_cap.latest,
                 'vol':std_dev,
                 'Price_Momentum_1M': Price_Momentum_1M,
                 'Earnings_Quality': cfs.operating_cash_flow.latest / EarningsSurprises.eps_act.latest,
                 'Price_Oscillator': Price_Oscillator,
        }, screen=base_universe
    
    )

In [15]:
start = pd.Timestamp("2018-01-01")
end = pd.Timestamp("2018-02-01")
end_1m = pd.Timestamp("2018-02-20")
test_start = pd.Timestamp("2018-03-01")

In [16]:
start_timer = time()
results = run_pipeline(make_pipeline(), start,end)
end_timer = time()
print("Time to run pipeline %.2f secs" %(end_timer - start_timer))

Time to run pipeline 10.94 secs


In [17]:
results.index.names = ['date', 'security']

In [18]:
results.head()

Earnings_Quality  \
date                      security                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])       6.880000e+08   
                          Equity(24 [AAPL])      7.563285e+09   
                          Equity(53 [ABMD])      9.808409e+07   
                          Equity(62 [ABT])       3.037879e+09   
                          Equity(67 [ADSK])      1.216667e+08   

                                             Price_Momentum_1M  \
date                      security                               
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])            1.108581   
                          Equity(24 [AAPL])           0.985735   
                          Equity(53 [ABMD])           0.962117   
                          Equity(62 [ABT])            1.013482   
                          Equity(67 [ADSK])           0.956391   

                                             Price_Oscillator  \
date                      security                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])           0.003372   
                          Equity(24 [AAPL])          0.150833   
                          Equity(53 [ABMD])          0.296576   
                          Equity(62 [ABT])           0.162189   
                          Equity(67 [ADSK])          0.037483   

                                             assets_turnover  cf_yield  \
date                      security                                       
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])          0.168792    0.0623   
                          Equity(24 [AAPL])         0.145953    0.0716   
                          Equity(53 [ABMD])         0.197232    0.0178   
                          Equity(62 [ABT])          0.095245    0.0532   
                          Equity(67 [ADSK])         0.121162   -0.0027   

                                             dividend_yield  \
date                      security                            
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])           0.0088   
                          Equity(24 [AAPL])          0.0145   
                          Equity(53 [ABMD])             NaN   
                          Equity(62 [ABT])           0.0186   
                          Equity(67 [ADSK])          0.0004   

                                             forward_earning_yeild    gearing  \
date                      security                                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])                  0.0550   1.152824   
                          Equity(24 [AAPL])                 0.0671   0.862981   
                          Equity(53 [ABMD])                 0.0179   0.026387   
                          Equity(62 [ABT])                  0.0490   0.750031   
                          Equity(67 [ADSK])                 0.0110  14.775396   

                                                gp_ta    market_cap  \
date                      security                                    
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])   0.031710  1.311608e+10   
                          Equity(24 [AAPL])  0.053104  8.608825e+11   
                          Equity(53 [ABMD])  0.160074  8.286548e+09   
                          Equity(62 [ABT])   0.054977  9.933608e+10   
                          Equity(67 [ADSK])  0.105443  2.309057e+10   

                                             net_margin    pb_ratio       roe  \
date                      security                                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])     0.031211    2.226839  0.017456   
                          Equity(24 [AAPL])    0.203770    6.422244  0.080414   
                          Equity(53 [ABMD])    0.184479   13.699354  0.041816   
                          Equity(62 [ABT])     0.088300    3.101151  0.018952   
                          Equity(67 [ADSK])   -0.232486  215.196374 -0.566698   

                                                  vol  
date               

In [19]:
start_timer = time()
assets = results.index.levels[1].unique()
pricing = get_pricing(assets, start_date=start, end_date=end_1m, fields='price')
end_timer = time()
print("Time to extract prices %.2f secs" %(end_timer-start_timer))

Time to extract prices 0.83 secs


In [20]:
# historical Pricing data
pricing.head()

,Equity(2 [ARNC]),Equity(24 [AAPL]),Equity(53 [ABMD]),Equity(62 [ABT]),Equity(67 [ADSK]),Equity(76 [TAP]),Equity(114 [ADBE]),Equity(122 [ADI]),Equity(128 [ADM]),Equity(161 [AEP]),...,Equity(51016 [JBGS]),Equity(51046 [BHF]),Equity(51079 [RDFN]),Equity(51091 [VNTR]),Equity(51157 [DWDP]),Equity(51205 [BKI]),Equity(51231 [ROKU]),Equity(51268 [SWCH]),Equity(51494 [DLPH]),Equity(51689 [PAGS])
2018-01-02 00:00:00+00:00,27.705,171.551,192.43,58.510,107.10,82.25,177.69,90.31,39.769,71.743,...,34.54,57.67,31.04,22.85,71.89,45.900,52.011,17.17,54.898,NaN
2018-01-03 00:00:00+00:00,27.914,171.481,195.77,58.630,109.36,81.46,180.98,91.37,39.481,71.109,...,34.46,58.52,28.17,23.11,73.03,45.750,55.380,17.28,53.792,NaN
2018-01-04 00:00:00+00:00,29.261,172.317,199.30,58.540,112.09,82.05,183.21,91.29,40.136,70.277,...,33.68,59.50,27.30,23.01,74.45,46.900,51.169,17.26,56.264,NaN
2018-01-05 00:00:00+00:00,29.541,174.260,202.28,58.709,110.82,82.93,185.35,91.64,39.858,70.099,...,33.63,60.61,28.19,23.47,75.36,46.850,46.250,17.58,56.294,NaN
2018-01-08 00:00:00+00:00,29.691,173.632,207.80,58.560,111.41,83.87,185.02,91.84,39.769,70.733,...,33.74,60.49,27.68,23.62,75.14,48.775,47.450,17.98,55.676,NaN


In [23]:
# calculate the 1 month forward returns predictions

df = pricing.pct_change(n_fwd_days).shift(-n_fwd_days)
df = df[:end]
df.head()

,Equity(2 [ARNC]),Equity(24 [AAPL]),Equity(53 [ABMD]),Equity(62 [ABT]),Equity(67 [ADSK]),Equity(76 [TAP]),Equity(114 [ADBE]),Equity(122 [ADI]),Equity(128 [ADM]),Equity(161 [AEP]),...,Equity(51016 [JBGS]),Equity(51046 [BHF]),Equity(51079 [RDFN]),Equity(51091 [VNTR]),Equity(51157 [DWDP]),Equity(51205 [BKI]),Equity(51231 [ROKU]),Equity(51268 [SWCH]),Equity(51494 [DLPH]),Equity(51689 [PAGS])
2018-01-02 00:00:00+00:00,0.086446,-0.026418,0.312685,0.062382,0.079458,0.015684,0.122067,0.014506,0.059393,-0.063644,...,-0.043109,0.111479,-0.365657,-0.018600,0.022256,0.078976,-0.232855,-0.036692,0.013261,NaN
2018-01-03 00:00:00+00:00,0.042846,-0.068357,0.267355,0.051509,0.020849,-0.005524,0.080948,-0.024625,0.040728,-0.056547,...,-0.040917,0.061347,-0.298900,-0.090437,-0.031357,0.067760,-0.270134,-0.080150,0.009072,NaN
2018-01-04 00:00:00+00:00,-0.093674,-0.095533,0.201054,0.003075,-0.025872,-0.053016,0.038535,-0.064629,0.003214,-0.063278,...,-0.048990,-0.006218,-0.279121,-0.125598,-0.089322,0.018124,-0.215736,-0.097914,-0.082006,NaN
2018-01-05 00:00:00+00:00,-0.144240,-0.068071,0.147321,0.002913,0.016243,-0.059568,0.049474,-0.033828,0.045562,-0.082797,...,-0.049955,-0.030193,-0.217453,-0.094163,-0.046178,0.034152,-0.094054,-0.141069,-0.050165,NaN
2018-01-08 00:00:00+00:00,-0.154626,-0.085007,0.150192,0.001878,-0.007719,-0.092166,0.039671,-0.070557,0.052654,-0.097875,...,-0.058091,-0.024632,-0.169436,-0.067316,-0.060687,-0.004613,-0.036059,-0.167964,-0.029025,NaN


In [24]:
df1 = df.stack()
df1

2018-01-02 00:00:00+00:00  Equity(2 [ARNC])        0.086446
                           Equity(24 [AAPL])      -0.026418
                           Equity(53 [ABMD])       0.312685
                           Equity(62 [ABT])        0.062382
                           Equity(67 [ADSK])       0.079458
                           Equity(76 [TAP])        0.015684
                           Equity(114 [ADBE])      0.122067
                           Equity(122 [ADI])       0.014506
                           Equity(128 [ADM])       0.059393
                           Equity(161 [AEP])      -0.063644
                           Equity(166 [AES])       0.069404
                           Equity(168 [AET])       0.040870
                           Equity(185 [AFL])       0.019552
                           Equity(197 [AGCO])      0.022740
                           Equity(216 [HES])       0.057483
                           Equity(239 [AIG])       0.077983
                           Equity(266 [A

In [25]:
df1.index.names = ['date', 'security']

In [26]:
df1

date                       security            
2018-01-02 00:00:00+00:00  Equity(2 [ARNC])        0.086446
                           Equity(24 [AAPL])      -0.026418
                           Equity(53 [ABMD])       0.312685
                           Equity(62 [ABT])        0.062382
                           Equity(67 [ADSK])       0.079458
                           Equity(76 [TAP])        0.015684
                           Equity(114 [ADBE])      0.122067
                           Equity(122 [ADI])       0.014506
                           Equity(128 [ADM])       0.059393
                           Equity(161 [AEP])      -0.063644
                           Equity(166 [AES])       0.069404
                           Equity(168 [AET])       0.040870
                           Equity(185 [AFL])       0.019552
                           Equity(197 [AGCO])      0.022740
                           Equity(216 [HES])       0.057483
                           Equity(239 [AIG])       0

In [31]:
df1.name = '1month_predicted_returns'

In [35]:
df_combined = results.join(df1)
df_combined.head(5)

Earnings_Quality  \
date                      security                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])       6.880000e+08   
                          Equity(24 [AAPL])      7.563285e+09   
                          Equity(53 [ABMD])      9.808409e+07   
                          Equity(62 [ABT])       3.037879e+09   
                          Equity(67 [ADSK])      1.216667e+08   

                                             Price_Momentum_1M  \
date                      security                               
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])            1.108581   
                          Equity(24 [AAPL])           0.985735   
                          Equity(53 [ABMD])           0.962117   
                          Equity(62 [ABT])            1.013482   
                          Equity(67 [ADSK])           0.956391   

                                             Price_Oscillator  \
date                      security                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])           0.003372   
                          Equity(24 [AAPL])          0.150833   
                          Equity(53 [ABMD])          0.296576   
                          Equity(62 [ABT])           0.162189   
                          Equity(67 [ADSK])          0.037483   

                                             assets_turnover  cf_yield  \
date                      security                                       
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])          0.168792    0.0623   
                          Equity(24 [AAPL])         0.145953    0.0716   
                          Equity(53 [ABMD])         0.197232    0.0178   
                          Equity(62 [ABT])          0.095245    0.0532   
                          Equity(67 [ADSK])         0.121162   -0.0027   

                                             dividend_yield  \
date                      security                            
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])           0.0088   
                          Equity(24 [AAPL])          0.0145   
                          Equity(53 [ABMD])             NaN   
                          Equity(62 [ABT])           0.0186   
                          Equity(67 [ADSK])          0.0004   

                                             forward_earning_yeild    gearing  \
date                      security                                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])                  0.0550   1.152824   
                          Equity(24 [AAPL])                 0.0671   0.862981   
                          Equity(53 [ABMD])                 0.0179   0.026387   
                          Equity(62 [ABT])                  0.0490   0.750031   
                          Equity(67 [ADSK])                 0.0110  14.775396   

                                                gp_ta    market_cap  \
date                      security                                    
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])   0.031710  1.311608e+10   
                          Equity(24 [AAPL])  0.053104  8.608825e+11   
                          Equity(53 [ABMD])  0.160074  8.286548e+09   
                          Equity(62 [ABT])   0.054977  9.933608e+10   
                          Equity(67 [ADSK])  0.105443  2.309057e+10   

                                             net_margin    pb_ratio       roe  \
date                      security                                              
2018-01-02 00:00:00+00:00 Equity(2 [ARNC])     0.031211    2.226839  0.017456   
                          Equity(24 [AAPL])    0.203770    6.422244  0.080414   
                          Equity(53 [ABMD])    0.184479   13.699354  0.041816   
                          Equity(62 [ABT])     0.088300    3.101151  0.018952   
                          Equity(67 [ADSK])   -0.232486  215.196374 -0.566698   

                                                  vol  \
date              